# Seq2Seq

*Sequence to Sequence*.  
*Encoder-Decoder Model*とも。

文章を入力とし、文章を出力するモデル。

In [1]:
from xml.etree import ElementTree as ET
from glob import glob
import os
from typing import List
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 発想

これまで、RNNを用いて、入力した単語列に続く単語を予測するモデルを作成し、単語の予測を繰り返すことで文章を生成した。

RNNは時系列の情報を保持するために隠れ状態$h_t$を用いる。隠れ状態の初期値$h_0$は0ベクトルとしているが、ここで、何らかの入力データから生成したベクトルを$h_0$として用いることを考える。このとき、上手く学習させれば、その入力に基づいた文章を生成できそう。

例えば、入力を画像をとし、CNNを用いて抽出した特徴量を$h_0$として用いるようにすれば、入力画像に基づいた文章が生成できる。画像のキャプションなどが例に挙げられる。  
学習方法は簡単で、入力画像に対して適切な文章が出力されるように学習させるだけ。隠れ状態を通じてRNNからCNNまで逆伝播を繋げる。

では、入力に文章を用いることはできないだろうか。RNNに文章を入力し、最後に出力された隠れ状態を文章ベクトルとする。これを別のRNNへの入力$h_0$とすれば、入力文に基づいた文章生成が可能になる。  

この発想は翻訳タスクに大きく役立つ。入力と出力に同じ意味を持った異なる言語の文章を設定すれば、入力文と同じ意味を持った文章生成が可能になる。  
こういったシーケンスをシーケンスに変換するモデルを *seq2seq (Sequence to Sequence)* モデルと呼ぶ。

本章では、入力に日本語文、出力に入力と同じ意味を持つ英語文を設定し、日本語→英語の翻訳を行うモデルを作成する。

このモデルは以下の二つのRNNから構成される。
- ***Encoder***: 文章ベクトルを生成するRNN
- ***Decoder***: 文章ベクトルを受け取って出力文を生成するRNN

このことから***Encoder-Decoderモデル***とも呼ばれる。


---

## データセット

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットを使用する。
- [Wikipedia日英京都関連文書対訳コーパス](https://alaginrc.nict.go.jp/WikiCorpus/index.html)

In [3]:
data_ja, data_en = [], []
root_dir = 'data/jaen-kyoto/'
for xml_file in glob(os.path.join(root_dir, '*/*.xml')):
    try:
        tree = ET.parse(xml_file)
    except ET.ParseError:
        continue
    root = tree.getroot()
    for sentence in root.iter('sen'):
        ja = sentence.find('j').text
        en = sentence.findall('e')[-1].text
        if ja and en:
            data_ja.append(ja)
            data_en.append(en)

# examples
for _ in range(5):
    i = random.randint(0, len(data_en))
    print('en:', data_en[i])
    print('ja:', data_ja[i])
    print()

n_data = len(data_en)
print('num of data:', n_data)

en: All the universe is substantial and has all the nine factors at its onset.
ja: 諸法の生ずる時は、体及び余の法は凡て九事有り。

en: In his later years he got acquainted with Shingoro TAKENO, a rich merchant in Sakai, and while Sanetaka taught TAKENO the study of poetry and waka poems, TAKENO gave him a lot of financial support.
ja: 晩年には堺の富商である武野新五郎と知己となり、実隆が武野に歌学や和歌を指導する一方、武野からは少なからず経済的援助を受けていた。

en: Himorogi in Koshinto considered shiniki as a place for a god to stay, or a border to the other world and this world, and was feared and respected.
ja: 古神道において神籬は、神の宿る場所としての神域、または常世（とこよ）と現世（うつしよ）の端境と考えれ、恐れ敬った。

en: Some people say that this is the first appearance of present seishu (refined sake), but it is controversial because of the following.
ja: これを以て現在の清酒（せいしゅ）の初見とみなす説があるが、それは以下のように議論の分かれるところである。

en: After appointment to Shogoinoge (Senior Fifth Rank, Lower Grade) the next year, he became Kurodo (Chamberlain) as well as Bicchu gon no suke (acting assistant governor of Bicchu Province) in 1168 and resign

多すぎるので減らす。

In [4]:
n_data = 10000
data_ja = data_ja[:n_data]
data_en = data_en[:n_data]

書き出し

In [5]:
textfile_ja = 'data/kyoto_ja.txt'
with open(textfile_ja, 'w') as f:
    f.write('\n'.join(data_ja))

textfile_en = 'data/kyoto_en.txt'
with open(textfile_en, 'w') as f:
    f.write('\n'.join(data_en))

### 前処理

トークナイザの学習

In [ ]:
pad_id = 3

vocab_size_ja = 8000
tokenizer_prefix_ja = 'models/tokenizer_kyoto_ja'
spm.SentencePieceTrainer.Train(
    input=textfile_ja, # データセット
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size_ja,
    pad_id=pad_id
)

vocab_size_en = 8000
tokenizer_prefix_en = 'models/tokenizer_kyoto_en'
spm.SentencePieceTrainer.Train(
    input=textfile_en, # データセット
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size_en,
    pad_id=pad_id
)

In [7]:
sp_ja = spm.SentencePieceProcessor(f'{tokenizer_prefix_ja}.model')
sp_en = spm.SentencePieceProcessor(f'{tokenizer_prefix_en}.model')
n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print('num of vocabrary (ja):', n_vocab_ja)
print('num of vocabrary (en):', n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [8]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [9]:
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)


---

## 学習データ

入力文と正解のペアを作成する。

Encoderへの入力（入力文）とDecoderの出力（正解）のペアを作成する。  
また、Decoderへの入力を考える必要がある。今回は教師強制を採用し、出力文の頭に\<BOS>を付与したものをDecoderへの入力とする。

例）
Encoderへの入力（入力文） | Decoderへの入力 | Eecoderの出力（出力文）
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

DataLoaderを作成する。  
**まずバッチサイズ1で進める。**

In [10]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # encoderへの入力
        x_dec = en[:-1] # decoderへの入力
        y_dec = en[1:] # decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

batch_size = 1
dataset = TextDataset(data_ids_ja, data_ids_en)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
x_enc, x_dec, y_dec = next(iter(dataloader))
x_enc, x_dec, y_dec # example

(tensor([[   4,   26, 5721,  699,   27,    8,  200, 7995,   11,    6,   16,  738,
             8, 3041, 2198,  635,   12,   26, 5687,   14,  775, 7653,   27,    6,
           135,  379,   12,   26, 5687,   14,  775,   14,  783, 2720, 1128,  552,
           719, 1929, 2833,  824,  597,   12,  989,    7,    5,    2]]),
 tensor([[   1,   21,  449,  146,    7,   43, 5001,  500,   31,   15,   43, 3651,
           356, 2707, 4092,  220,  144,    4, 5172, 1955,   69,   20,    4,  225,
           715,    5,    8,   43, 3651,  356, 1310,  500,   31,   23,    4, 4513,
             7,    4,  281,   12, 3346,   45,  952,   38,   46,  194, 3711,    9,
           511, 2369,   95,  270,   54]]),
 tensor([[  21,  449,  146,    7,   43, 5001,  500,   31,   15,   43, 3651,  356,
          2707, 4092,  220,  144,    4, 5172, 1955,   69,   20,    4,  225,  715,
             5,    8,   43, 3651,  356, 1310,  500,   31,   23,    4, 4513,    7,
             4,  281,   12, 3346,   45,  952,   38,   46,  194, 


---

## モデル構築

EncoderとDecoderを作る。

### Encoder

入力文を入れて隠れ状態を出力するだけのRNN。RNN層と線形層で作る。

In [11]:
class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        _, h = self.rnn(x) # h: (1, batch_size, hidden_size)
        h = self.fc(h) # (1, batch_size, hidden_size)
        return h

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、RNN層と線形層で作る。



In [12]:
class Decoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h):
        x = self.embedding(x) # (seq_len, embed_size)
        y, h = self.rnn(x, h) # y: (seq_len, hidden_size), h: (1, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, h

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [13]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        y, _ = self.decoder(x_dec, h)
        return y

### 学習

In [14]:
def train(model, optimizer, criterion, n_epochs, prog_unit=1):
    model.train()
    prog.start(n_iter=len(dataloader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        for x_enc, x_dec, y_dec in dataloader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y_pred = model(x_enc, x_dec)
            loss = criterion(y_pred.reshape(-1, n_vocab_ja), y_dec.ravel())
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

In [15]:
hidden_size, embed_size = 1024, 1024
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

一旦動くことが確認できれば良いので、エポック数1で学習させる。

In [16]:
train(model, optimizer, criterion, n_epochs=1)

1/1: ##                                         6% [00:00:08.58] loss: 6.49081 

KeyboardInterrupt: 


---

## ミニバッチ学習

ミニバッチ学習に対応させる。

まずはRMMLMのときのようにバッチ内のサイズが揃うようにpaddingをする。

In [17]:
def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

batch_size = 32
dataset = TextDataset(data_ids_ja, data_ids_en)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)

損失関数はpadトークンを無視するように変更する。

In [18]:
criterion = nn.CrossEntropyLoss(ignore_index=pad_id)

次に、1つ工夫を加える。このままでも動きはするが、学習が上手くいかない可能性がある。

encoderへの入力はpaddingされたデータである。ここで、paddingされた範囲が多い=padトークンが多く含まれているデータは、padの数が多くなるにつれて、隠れ状態がある一定の値に収束してしまう。RNNに同じトークンを何度も入力することで隠れ状態が収束していってしまうのだ。  
そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、翻訳を学習できない。

そこで、encoderが出力する隠れ状態として、padトークンを除いた最後のトークン=EOSを入力した時点のものを使用する。  
encoderを以下のように変更する。

In [19]:
eos_id = sp_ja.eos_id()

class Encoder(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.rnn = nn.RNN(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_positions = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)
        y, _ = self.rnn(x) # y: (batch_size, seq_len, hidden_size)
        h = y[eos_positions] # (batch_size, hidden_size)
        h = self.fc(h).unsqueeze(0) # (1, batch_size, hidden_size)
        return h

ではこれで学習させてみよう。

In [20]:
hidden_size, embed_size = 1024, 1024
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [22]:
train(model, optimizer, criterion, n_epochs=100, prog_unit=10)

   1-10/100: ######################################## 100% [00:02:01.03] loss: 3.55206 
  11-20/100: ######################################## 100% [00:02:03.89] loss: 2.31923 
  21-30/100: ######################################## 100% [00:02:03.33] loss: 1.53096 
  31-40/100: ######################################## 100% [00:02:01.06] loss: 0.96194 
  41-50/100: ######################################## 100% [00:02:03.29] loss: 0.56233 
  51-60/100: ######################################## 100% [00:02:03.42] loss: 0.30002 
  61-70/100: ######################################## 100% [00:02:03.65] loss: 0.14746 
  71-80/100: ######################################## 100% [00:02:03.85] loss: 0.07586 
  81-90/100: ######################################## 100% [00:02:03.21] loss: 0.05850 
 91-100/100: ######################################## 100% [00:02:02.93] loss: 0.04216 



---

## 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。

In [71]:
unk_id = sp_en.unk_id() # UNKのID
def token_sampling(y: List[float]) -> int:
    """モデルの出力から単語をサンプリングする"""
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device)

    h = model.encoder(in_ids)
    next_token = bos_id

    token_ids = []
    for _ in range(max_len):
        x = torch.tensor([[next_token]], device=device)
        y, h = model.decoder(x, h)
        y = y[0]
        if decisive:
            next_token = y.argmax().item()
        else:
            next_token = token_sampling(y)
        token_ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = sp_en.decode(token_ids)
    return sentence

In [76]:
n = 5
for x, t in zip(data_ja[:n], data_en[:n]):
    print('input:', x)
    print('output:', translate(model, x))
    print('answer:', t)
    print()

input: 駅情報
output: Information
answer: Information

input: 三条京阪駅（さんじょうけいはんえき）は、京都市東山区にある、京都市営地下鉄東西線の鉄道駅。
output: Located in the Higashiyama Ward of Kyoto City, Sanjyo-Keihan Station is a stop on the Tozai Line, a Kyoto Municipal Subway Line.
answer: Located in the Higashiyama Ward of Kyoto City, Sanjyo-Keihan Station is a stop on the Tozai Line, a Kyoto Municipal Subway Line.

input: 駅番号はT11。
output: The station number is T11.
answer: The station number is T11.

input: 京阪電気鉄道
output: Keihan Electric Railway
answer: The Keihan Electric Railway

input: 京阪本線・京阪鴨東線（三条駅 (京都府)）
output: Keihan Main Line and Keihan Oto Line in Sanjyo Station (in Kyoto Prefecture)
answer: Keihan Main Line and Keihan Oto Line in Sanjyo Station (in Kyoto Prefecture)



In [77]:
sentences = [
    'この駅は京都市内の中心部にあります。',
    '京都'
]

In [78]:
for sentence in sentences:
    print('input:', sentence)
    print('output:', translate(model, sentence))
    print()

input: この駅は京都市内の中心部にあります。
output: Trains at Demachiyanagi Station, the train with no one-sided one-way--which considered to a seven-car train.

input: 京都
output: Nonomiya-jinja Shrine (via Gion)

